# Rendre inactifs 6264 acteurs reparation LVAO (ancien CMA)

[Ticket](https://www.notion.so/accelerateur-transition-ecologique-ademe/Supprimer-les-artisans-Ferm-s-de-la-vieille-source-CMA-non-r-paracteurs-12e6523d57d78045a70ed85ad8a47796)

 - **QUOI**: modifier 6264 acteurs LVAO `_reparation_` (anciennement CMA) en changeant `statut` = `INACTIF`
 - **POURQUOI**: les entrées dans la base sont obsolètes
 - **COMMENT**:
    - intégration avec la base SIRENE en local
    - génération d'un export et revue manuelle
    - génération des requettes SQL sur la base de l'export



## Paramètres

In [73]:
import os
from datetime import datetime, timezone

# Base de données
DB_TEST_URL = os.environ["DB_TEST_URL"]
DB_PROD_URL = os.environ["DB_PROD_URL"]
DB_TABLE = "qfdmo_acteur"
DB_TABLE_REVISION = "qfdmo_revisionacteur"
DB_STATUS_FIELD = "statut"
DB_STATUS_VALUE = "INACTIF"
DB_COMMENT_FIELD = "commentaires"
DB_COMMENT_VALUE = "20241104_acteurs_inactifs_reparation_lvao_ancien_cma"
DB_ID_FIELD = "identifiant_unique"
DB_TIMESTAMP_FIELD = "modifie_le"
# on ne touche pas à se qui a été modifié end prod
# depuis l'export
DB_TIMESTAMP_SAFEGUARD = datetime.strptime("2024-10-29 00:00:00", "%Y-%m-%d %H:%M:%S").replace(tzinfo=timezone.utc)

# File system
#DIR_CURRENT = os.path.dirname(os.path.abspath(__file__))
#DIR_DATA = os.path.join(DIR_CURRENT, "data")

# Export validé
DATA_FILEPATH = "./data/acteurs_a_desactiver.csv"
DATA_FILTER_KEY = "validation"
# note: seule un échantillion à été validé donc on garde
# tout sauf 1 entrée modifiée sur prod entre temps
DATA_FILTER_EXCLUDE = "NE PAS TOUCHER"
DATA_FILTER_LENGTH = 9653 # on s'attend à modifier 6264 acteurs
DATA_STATUS_FIELD = "statut"
DATA_STATUS_CURRENT = "ACTIF"

# Les champs de la source de données SIRENE nous indiquant
# que soit l'entreprise est en cessation (C), soit
# l'établissement est fermé (F)
DATA_SIREN_STATUS_FIELD = "etatAdministratifUniteLegale"
DATA_SIREN_STATUS_VALUE = "C"
DATA_SIRET_STATUS_FIELD = "etatAdministratifEtablissement"
DATA_SIRET_STATUS_VALUE = "F"


## Import des librairies

In [74]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from sqlalchemy.engine.url import make_url
from rich import print
from rich.progress import track

## Valider connection à la base de données

In [75]:
# Create and test engines
#assert DB_TEST_URL != DB_PROD_URL, "DB_TEST_URL et DB_PROD_URL doivent être différents"
db_test_engine = create_engine(DB_TEST_URL)
db_prod_engine = create_engine(DB_PROD_URL)
for engine in [db_test_engine, db_prod_engine]:
    conn = engine.connect()
    conn.execute(f"SELECT * FROM {DB_TABLE} LIMIT 1")
    conn.close()

## Lecture et validation de la data locale

In [76]:
# The default "df" corresponds to our local data
df = pd.read_csv(DATA_FILEPATH).replace({np.nan: None})
df.head()

,validation,Django,sirene_unite_legale,sirene_etablissement,etatAdministratifUniteLegale,etatAdministratifEtablissement,nom,description,siret,identifiant_unique,adresse,adresse_complement,code_postal,ville,statut
0,None,lien,lien,lien,A,F,Yannis Artisan Joaillier,None,50431909600013,yannis_artisan_joaillier_212320_reparation,18 PL NOTRE DAME,None,95300,PONTOISE,ACTIF
1,None,lien,lien,lien,A,F,Dt Menuiserie,None,85292292100022,dt_menuiserie_220790_reparation_0786288840,4 RUE DES BLEUETS,None,21490,RUFFEY LES ECHIREY,ACTIF
2,None,lien,lien,lien,A,F,La Foret D'Emeraude,None,88135152200027,la_foret_demeraude_205284_reparation_0621698122,2 RUE MARIE DURANT,LE CONCERTO APP C102,13640,LA ROQUE-D'ANTHÉRON,ACTIF
3,None,lien,lien,lien,A,F,Talos Informatique,None,84055429900015,talos_informatique_39282_reparation_0685647212,30 LA PLACETTE,None,7120,CHAUZON,ACTIF
4,None,lien,lien,lien,A,F,L Atelier De Boo,None,44879360400043,l_atelier_de_boo_205770_reparation_0674265167,3 RUE DES COMPAGNONS,None,27100,VAL DE REUIL,ACTIF


In [77]:
# On filtre les acteurs à modifier
filter_validation = df[DATA_FILTER_KEY] != DATA_FILTER_EXCLUDE
filter_siren = df[DATA_SIREN_STATUS_FIELD] == DATA_SIREN_STATUS_VALUE
filter_siret = df[DATA_SIRET_STATUS_FIELD] == DATA_SIRET_STATUS_VALUE

df = df[filter_validation & (filter_siren | filter_siret)]
print(f"Nombre d'acteurs à modifier: {len(df)}")
assert len(df) == DATA_FILTER_LENGTH, f"On devrait avoir {DATA_FILTER_LENGTH} acteurs à modifier"

Nombre d'acteurs à modifier: 9653

In [78]:
# On s'assure qu'ils étaient tous actifs
assert all(df[DATA_STATUS_FIELD] == DATA_STATUS_CURRENT), "Tous les acteurs devraient être actifs"
# Qu'on avait pas de doublon sur le champ identifiant
assert len(df) == len(df[DB_ID_FIELD].unique()), f"Présence de doublons sur le champ {DB_ID_FIELD}"

## Logique de modification de la base

In [84]:
def sql_acteurs_select(table_name: str, ids: list):
    """Construit une requête SQL pour récupérer des acteurs matchant des IDs"""
    return f"""SELECT {DB_ID_FIELD}, {DB_TIMESTAMP_FIELD}
            FROM {table_name}
            WHERE {DB_ID_FIELD} IN ({', '.join(["'"+str(i)+"'" for i in ids])})"""

def db_update_from_df(df: pd.DataFrame, engine):
    """MAJ de la DB à partir d'un DataFrame source d'acteurs"""
    url = make_url(engine.url)
    print("db_update_from_df:")
    print({"hostname": url.host,"port":url.port, "database": url.database})

    # On récupère les IDs de la dataframe source
    ids = df[DB_ID_FIELD].tolist()

    # On récupère les acteurs vs. révisions existants et on s'assure du match
    df_act = pd.read_sql(con=engine, sql=sql_acteurs_select(DB_TABLE, ids))
    df_rev = pd.read_sql(con=engine, sql=sql_acteurs_select(DB_TABLE_REVISION, ids))
    print({"# acteurs": len(df_act), "# révisions existantes": len(df_rev)})
    assert len(df_act) == len(df_rev), "Certaines révisions manquantes, à créer..."

    # On met à jour les révisions
    print("MAJ des révisions: début...")
    count=0
    for index, row in df_rev.iterrows():
        sql_update = f"""
            UPDATE {DB_TABLE_REVISION}
            SET {DB_STATUS_FIELD} = '{DB_STATUS_VALUE}',
                {DB_COMMENT_FIELD} = '{DB_COMMENT_VALUE}',
                {DB_TIMESTAMP_FIELD} = NOW()
            WHERE {DB_ID_FIELD} = '{row[DB_ID_FIELD]}'"""
        if index==0:
            print(sql_update)
        engine.execute(sql_update)
        count+=1
        if count%500==0:
            print(count)
    print("MAJ des révisions: fin")
    print("# entrées modifiées:", count)

## Modification de la base

### Test

In [85]:
db_update_from_df(df, db_test_engine)

db_update_from_df:

{'hostname': 'localhost', 'port': 6543, 'database': 'qfdmo'}

{'# acteurs': 9653, '# révisions existantes': 9653}

MAJ des révisions: début...

UPDATE qfdmo_revisionacteur
            SET statut = 'INACTIF',
                commentaires = '20241104_acteurs_inactifs_reparation_lvao_ancien_cma',
                modifie_le = NOW()
            WHERE identifiant_unique = 'ma2j_108629_reparation'

...

8500

9000

9500

MAJ des révisions: fin

# entrées modifiées: 9653

## Production

In [86]:
raise Exception("Sur la point de modifier la base de prod, retirer cette ligne pour continuer")

In [87]:
#db_update_from_df(df, db_prod_engine)

db_update_from_df:

{
    'hostname': 'quefairedem-5084.postgresql.a.osc-fr1.scalingo-dbs.com',
    'port': 33262,
    'database': 'quefairedem_5084'
}

{'# acteurs': 9653, '# révisions existantes': 9653}

MAJ des révisions: début...

UPDATE qfdmo_revisionacteur
            SET statut = 'INACTIF',
                commentaires = '20241104_acteurs_inactifs_reparation_lvao_ancien_cma',
                modifie_le = NOW()
            WHERE identifiant_unique = 'fps_171567_reparation'

...

8500

9000

9500

MAJ des révisions: fin

# entrées modifiées: 9653